<a href="https://colab.research.google.com/github/leandrominer85/Scrap_sites_noticias/blob/main/Scrap_sites_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Scrap de sites de notícias**

## Esse projeto tem como objetivo fazer a raspagem de dados de alguns sites de notícias. Esses scripts permitem fazer a raspagem dos dados das principais notícias tanto da home page quanto das 'últimas notícias'. Quando disponível utilizaremos o feed RSS.

## Os sites que serão utilizados são : g1, r7, folha, estadão, metro, globo, uol, terra, msn, buzzfeed, veja, ig, antagonista, oglobo, ny_times, washington, cnn_br, cnn_en, bbc_br e bbc_en.

## As saídas do projeto são  arquivos .xlsx para cada um dos sites com as seguintes colunas : "fonte", 'Título', 'Subtítulo', 'Link' e 'Hora'.

Para rodar o script no colab são necessárias algumas instalações de módulos (selenium e chromer driver):

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

Agora podemos instanciar o chromedriver que será utilizado por todos os scripts que utilizam selenium:

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


Importação dos módulos:

In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
from selenium.webdriver.chrome.options import Options
from urllib.request import urlopen, Request
from selenium.webdriver.common.keys import Keys
from multiprocessing import Process
import warnings
warnings.filterwarnings("ignore")

Temos 3 tipos de extração de dados: extração direta pelo beautifulsoup, extração utilizando o selenium + o beautifulsoup e extração por RSS.
Mostrarei um exemplo com comentarios para cada uma das extrações nas próximas 2 funções. As demais seguem uma ou mais das extrações apenas com modificações relativas ao site e formatação.

In [4]:
def g1():
    #Primeira parte: utilizando o BeautifulSoup para extração da homepage
    
    #criando uma lista vazia para receber as notícias
    lista_noticias = []
    
    #Obtendo a resposta pela biblioteca requests
    response = requests.get('https://g1.globo.com/')
    
    #Obtendo o conteúdo da resposta
    content = response.content
    
    #Utilizando o BeautifulSoup para adquirir o conteúdo
    site = BeautifulSoup(content, 'html.parser')

    # HTML das notícias (procurando por todas as notícias com os dados que queremos)
    noticias = site.findAll('div', attrs={'class': 'feed-post-body'})

    #Iterando sobre as notícias
    for noticia in noticias:
      # Título
        titulo = noticia.find('a', attrs={'class': 'feed-post-link'})

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = noticia.find('div', attrs={'class': 'feed-post-body-resumo'})

        #O subtítulo nem sempre está presente, por isso é necessário esse if-else
        if (subtitulo):
            
            lista_noticias.append(["g1_home",titulo.text, subtitulo.text, titulo['href'],
                                   datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])
        else:
            lista_noticias.append(["g1_home",titulo.text, '', titulo['href'],
                                   datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])


    # Transformando a lista em dataframe
    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link', "Hora"])
    
    
    # Segunda parte da extração - utilizando o RSS
    
    #Reinstanciando a lista de noticias
    lista_noticias = []
    url = "http://g1.globo.com/dynamo/rss2.xml"
    
    #Obtendo a resposta pela biblioteca requests
    response = requests.get(url)
    
    #Obtendo o conteúdo da resposta
    site = BeautifulSoup(response.content, features='xml')
    
    # HTML das notícias (procurando por todas as notícias com os dados que queremos)
    noticias = site.findAll('item')

    #Iterando sobre as notícias
    for noticia in noticias:
      # Título
        titulo = noticia.find('title')
        
        #O subtítulo nem sempre está presente, por isso é necessário esse if-else
        subtitulo = re.sub(r'<.+?>', '', noticia.find('description').text).strip()

        if (subtitulo):

            #Adiciono uma coluna com o nome da fonte, o título, o subtítulo e a hora do acesso com o módulo time
            lista_noticias.append(["g1_ultimas", titulo.text, subtitulo,
                                   noticia.find("link").text, datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])
        else:
            lista_noticias.append(["g1_ultimas" ,titulo.text, '',
                                   noticia.find("link").text, datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])
            
    # Transformando a lista em dataframe
    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link', "Hora"])
    
    #Concatenando os dois dataframes
    df = pd.concat([news1,news2])
    
    #Salvando o dataframe
    df.to_excel('g1.xlsx', index=False)

In [5]:
def buzz():
    #o site utiliza javascript, por isso é preciso utilizar a biblioteca selenium
    # Refs: https://medium.com/data-hackers/web-scraping-com-python-para-pregui%C3%A7osos-unindo-beautifulsoup-e-selenium-parte-2-8cfebf4f34e
    # https://blog.4linux.com.br/web-scraping-python-selenium-e-beautifulsoup/

    #Iniciando a lista vazia
    lista_noticias= []
    
    url = ("https://www.buzzfeednews.com/")

    #acessando a url (utilizando o driver definido acima)
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)

    #Procurando o elemento da página
    element = driver.find_element_by_class_name('splash-regular-stories')

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")




    # Neste site há quatro tipos de noticias mas todas seguem o esquema da principal
    
    #Obtendo as noticias
    principal = site.find('a',
                          attrs={'class': "splash-regular-stories__primary-story-image-link xs-block splash-regular-stories__link"})

    #Buscando o título
    titulo = site.find('h2',attrs={'class': "splash-regular-stories__primary-story-title"}).text.strip()

    #Apendando à lista
    lista_noticias.append(["buzz_home",titulo, '', principal['href'],
                           datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])


    secundaria = site.find('a',
                           attrs={'class': "splash-regular-stories__link xs-flex-align-start xs-flex-justify-space-between"})

    titulo= site.find('h2',attrs={'class': "splash-regular-stories__secondary-story-title"}).text.strip()

    lista_noticias.append(["buzz_home",titulo, '', secundaria['href'],
                           datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])


    terciaria = site.find('a',
                          attrs={'class': "xs-flex splash-regular-stories__link xs-flex-align-start xs-flex-justify-space-between"})

    titulo= site.find('h2',attrs={'class': "splash-regular-stories__tertiary-story-title xs-pr1 md-pr0"}).text.strip()

    lista_noticias.append(["buzz_home",titulo, '', terciaria['href'],
                           datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])


    #Procurando o elemento da página
    element = driver.find_element_by_class_name('sm-mt2')

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    # HTML da notícia
    noticias = site.findAll('a', attrs={'class': "newsblock-story-card__link xs-flex"})

    #Iterando sobre as notícias
    for noticia in noticias:
      # Título
        titulo = noticia.text


        #Populando a lista com os resultados
        lista_noticias.append(["buzz_home",titulo, '', noticia['href'],
                               datetime.now().strftime(("%d/%m/%Y %H:%M:%S"))])

    #Transformando a lista em Dataframe e salvando como .xlsx 
    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link', "Hora"])
    
    news1.to_excel('buzz.xlsx', index=False)

In [6]:
def r7():
    

    lista_noticias = []

    response = requests.get('https://www.r7.com/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'r7-flex-hat'})

    for noticia in noticias:
        try:
          # Título
            titulo = noticia.find('a', attrs={'class': 'r7-flex-hat__description'})

            # Subtítulo: div class="feed-post-body-resumo"
            subtitulo = titulo['title']

            if (subtitulo):
                # print(subtitulo.text)
                lista_noticias.append(['r7_home',titulo.text, subtitulo, titulo['href']])
            else:
                lista_noticias.append(['r7_home',titulo.text, '', titulo['href']])

        except:
            pass


    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])


    
    
    
    lista_noticias = []
    url = "https://noticias.r7.com/feed.xml"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('entry')

    for noticia in noticias:
      # Título
        titulo = noticia.find('title').text

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = re.sub(r'<.+?>', '', noticia.find('content').text).strip()

        if (subtitulo):

            lista_noticias.append(["r7_ultimas", titulo, subtitulo, noticia.find("link")['href']])
        else:
            lista_noticias.append(["r7_ultimas" ,titulo, '', noticia.find("link")['href']])
    
    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])
    
    df = pd.concat([news1,news2])
    
    df.to_excel('r7.xlsx', index=False)

In [7]:
def folha():
    lista_noticias = []

    response = requests.get('https://www.folha.uol.com.br/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'row'})

    for noticia in noticias:
      # Título
        try:
            titulo = noticia.find('h2', attrs={'class': "c-main-headline__title"}).text.strip()
        except:
            pass
        try:
            titulo = noticia.find('h2', attrs={'class': "c-headline__title"}).text.strip()
        except:
            continue


        subtitulo = noticia.find('p', attrs={'class': 'c-headline__standfirst'})

        if (subtitulo):

            lista_noticias.append(["folha_home",titulo, subtitulo.text, noticia.find('a')['href']])
        else:
            lista_noticias.append(["folha_home",titulo, '', noticia.find('a')['href']])


    news1 = pd.DataFrame(lista_noticias,
                         columns=["fonte",'Título', 'Subtítulo', 'Link']).drop_duplicates(subset= "Título",keep='last')

    lista_noticias = []
    url = "http://feeds.folha.uol.com.br/emcimadahora/rss091.xml"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('item')

    for noticia in noticias:
      # Título
        titulo = noticia.find('title')

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = re.sub(r'<.+?>', '', noticia.find('description').text).strip()

        if (subtitulo):

            lista_noticias.append(["folha_ultimas", titulo.text, subtitulo, noticia.find("link").text])
        else:
            lista_noticias.append(["folha_ultimas" ,titulo.text, '', noticia.find("link").text])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    df = pd.concat([news1,news2])

    df.to_excel('folha.xlsx', index=False)

In [8]:
def estadao():
    lista_noticias = []

    response = requests.get('https://www.estadao.com.br/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'intro'})

    for noticia in noticias:
      # Título
        try:
            titulo = noticia.find('h3', attrs={'class': "title"}).text.strip()
        except:
            continue


        subtitulo = noticia.find('p', attrs={'class': 'c-headline__standfirst'})

        if (subtitulo):

            lista_noticias.append(["estadao_home",titulo, subtitulo.text, noticia.find('a')['href']])
        else:
            lista_noticias.append(["estadao_home",titulo, '', noticia.find('a')['href']])


    news1 = pd.DataFrame(lista_noticias,columns=["fonte",'Título', 'Subtítulo', 'Link'])


    lista_noticias = []

    response = requests.get('https://www.estadao.com.br/ultimas')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'row'})

    for noticia in noticias:
      # Título
        try:
            titulo = noticia.find('h3', attrs={'class': "third"}).text.strip()
        except:
            continue


        subtitulo = noticia.find('p', attrs={'class': 'c-headline__standfirst'})

        if (subtitulo):

            lista_noticias.append(["estadao_ultimas",titulo, subtitulo.text, noticia.find('a')['href']])
        else:
            lista_noticias.append(["estadao_ultimas",titulo, '', noticia.find('a')["href"]])


    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo',
                                                  'Link']).drop_duplicates(subset= "Título",keep='last')

    df = pd.concat([news1,news2])

    df.to_excel('estadao.xlsx', index=False)

In [9]:
def metro():
    lista_noticias = []
    
    response = requests.get('https://www.metropoles.com/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'm-box-text'})

    for noticia in noticias:
      # Título
        try:
            titulo = noticia.find_all('a')[1].text
        except:
                continue

        try:
            subtitulo = noticia.find("p").text
        except:
            subtitulo = None

        if (subtitulo):

            lista_noticias.append(["metro_home",titulo, subtitulo, noticia.find_all('a')[1]['href']])
        else:
            lista_noticias.append(["metro_home",titulo, '', noticia.find_all('a')[1]['href']])


    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])



    lista_noticias = []

    response = requests.get('https://www.metropoles.com/ultimas-noticias')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'column is-full is-full'})

    for noticia in noticias:
      # Título
        try:
            titulo = noticia.find_all('a')[2].text
        except:
            continue


        subtitulo = noticia.find('p', attrs={'class': 'm-resume'}).text

        if (subtitulo):

            lista_noticias.append(["metro_ultimas",titulo, subtitulo, noticia.find('a')['href']])
        else:
            lista_noticias.append(["metro_ultimas",titulo, '', noticia.find('a')["href"]])


    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo',
                                                  'Link']).drop_duplicates(subset= "Título",keep='last')

    df = pd.concat([news1,news2])

    df.to_excel('metropoles.xlsx', index=False)

In [10]:
def globo():
    #o site utiliza javascript, por isso é preciso utilizar a biblioteca selenium
    # Refs: https://medium.com/data-hackers/web-scraping-com-python-para-pregui%C3%A7osos-unindo-beautifulsoup-e-selenium-parte-2-8cfebf4f34e
    # https://blog.4linux.com.br/web-scraping-python-selenium-e-beautifulsoup/
    
    

    url = ("https://www.globo.com/")
    
    #acessando a url
    driver.get(url)
    
    #Esperando o carregamento
    time.sleep(10)

    #Procurando o elemento da página
    element = driver.find_element_by_class_name('area-destaque')
    
    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")

    lista_noticias=[]

    # HTML da notícia
    noticias = site.findAll('a', attrs={'class': "post__link"})

    for noticia in noticias:
      # Título
        titulo = noticia['title']

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = noticia.text

        if (subtitulo):
            # print(subtitulo.text)
            lista_noticias.append(["globo_home",titulo, subtitulo, noticia['data-tracking-label']])
        else:
            lista_noticias.append(["globo_home",titulo, '', noticia['data-tracking-label']])


    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    lista_noticias = []
    url = "https://oglobo.globo.com/rss.xml?completo=true"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('item')

    for noticia in noticias:
      # Título
        titulo = noticia.find('title')

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = re.sub(r'<.+?>', '', noticia.find('description').text).strip()

        if (subtitulo):

            lista_noticias.append(["globo_ultimas", titulo.text, subtitulo, noticia.find("link").text])
        else:
            lista_noticias.append(["globo_ultimas" ,titulo.text, '', noticia.find("link").text])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    df = pd.concat([news1,news2])

    df.to_excel('globo.xlsx', index=False)

In [11]:
def uol():
    lista_noticias = []

    response = requests.get('https://noticias.uol.com.br/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')


    # HTML da notícia
    high = site.findAll('div', attrs={'class': 'highlights'})

    for h in high:
      # Título
        titulo = h.text

        lista_noticias.append(["uol_home",titulo, '', h.find("a")['href']])



    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'thumbnails-wrapper'})

    for noticia in noticias:
      # Título
        titulo = noticia.text

        lista_noticias.append(["uol_home",titulo, '', noticia.find("a")['href']])


    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    lista_noticias = []
    url = "http://rss.uol.com.br/feed/noticias.xml"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('item')

    for noticia in noticias:
      # Título
        titulo = noticia.find('title')

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = re.sub(r'<.+?>', '', noticia.find('description').text).strip()

        if (subtitulo):

            lista_noticias.append(["uol_ultimas", titulo.text, subtitulo, noticia.find("link").text])
        else:
            lista_noticias.append(["uol_ultimas" ,titulo.text, '', noticia.find("link").text])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    df = pd.concat([news1,news2])

    df.to_excel('uol.xlsx', index=False)

In [12]:
def terra():
    lista_noticias = []

    response = requests.get('https://www.terra.com.br/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('a', attrs={'class': 'card-news__text--title main-url'})

    for noticia in noticias:
      # Título
        titulo = noticia.text

        lista_noticias.append(["terra_home",titulo, '', noticia['href']])


    # HTML da notícia
    noticias = site.findAll('a', attrs={'class': 'card-h-small__text--title main-url'})

    for noticia in noticias:
      # Título
        titulo = noticia.text

        lista_noticias.append(["terra_home",titulo, '', noticia['href']])



    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])



    lista_noticias = []

    response = requests.get('https://www.terra.com.br/noticias/')
    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # HTML da notícia
    noticias = site.findAll('a', attrs={'class': 'card-news__text--title main-url'})

    for noticia in noticias:
      # Título
        titulo = noticia.text

        lista_noticias.append(["terra_noticias",titulo, '', noticia['href']])


    # HTML da notícia
    noticias = site.findAll('a', attrs={'class': 'card-h-small__text--title main-url'})

    for noticia in noticias:
      # Título
        titulo = noticia.text

        lista_noticias.append(["terra_noticias",titulo, '', noticia['href']])



    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])


    df = pd.concat([news1,news2])

    df.to_excel('terra.xlsx', index=False)

In [13]:
def msn():
    

    lista_noticias = []
    url = "https://rss.msn.com/pt-br/"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('item')

    for noticia in noticias:
      # Título
        titulo = noticia.find('title')


        lista_noticias.append(["msn_ultimas" ,titulo.text, '', noticia.find("link").text])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    news2.to_excel('msn.xlsx', index=False)

In [14]:
def veja():
    #o site utiliza javascript, por isso é preciso utilizar a biblioteca selenium
    # Refs: https://medium.com/data-hackers/web-scraping-com-python-para-pregui%C3%A7osos-unindo-beautifulsoup-e-selenium-parte-2-8cfebf4f34e
    # https://blog.4linux.com.br/web-scraping-python-selenium-e-beautifulsoup/

      

  url = ("https://veja.abril.com.br/")

  #acessando a url
  driver.get(url)

  #Esperando o carregamento
  time.sleep(3)

  #Procurando o elemento da página
  if driver.find_element_by_xpath("/html/body/main/section[1]").get_attribute("innerHTML").find('block edicao-semana dark') != None:
    element = driver.find_element_by_xpath("//section[2]")
  else:
    element = driver.find_element_by_xpath("//section[1]")



  #Carregando o html
  html = element.get_attribute("innerHTML")

  #Utilizando o BeautifulSoup para acessar os dados
  site = BeautifulSoup(html, "html.parser")

  lista_noticias=[]

  if site.find('div', attrs={'class': "card d"}) == None:
    element = driver.find_element_by_xpath("//section[1]")
    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")
    
  principal = site.find('div', attrs={'class': "card d"})
  titulo = principal.find('h2').text
  subtitulo = principal.find('p').text
  lista_noticias.append(["veja_home",titulo, subtitulo, principal.find('a')['href']])


  secundarias = site.findAll('div', attrs={'class': "col-s-12 col-l-4"})

  for noticia in secundarias:
    titulo= noticia.find("h3").text


    lista_noticias.append(["veja_home",titulo, '', noticia.find('a')['href']])



  terciarias = site.findAll('a', attrs={'class': "link related-article"})

  for noticia in terciarias:


    titulo= noticia.text

    lista_noticias.append(["veja_home",titulo, subtitulo, noticia['href']])


  news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])




  url = ("https://veja.abril.com.br/ultimas-noticias/")

  #acessando a url
  driver.get(url)

  #Esperando o carregamento
  time.sleep(3)

  #Procurando o elemento da página
  element = driver.find_element_by_class_name('list')

  #Carregando o html
  html = element.get_attribute("innerHTML")

  #Utilizando o BeautifulSoup para acessar os dados
  site = BeautifulSoup(html, "html.parser")


  # HTML da notícia
  noticias =site.findAll('div', attrs={'class': "row"})

  for noticia in noticias:
    # Título
      titulo = noticia.find('h2',  attrs={'class': "title"}).text


      lista_noticias.append(["veja_ultimas",titulo, '', noticia.find('a')['href']])


  news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link']).drop_duplicates(subset= "Título",keep='last')


  news2.to_excel('veja.xlsx', index=False)


In [15]:
def ig():

    lista_noticias = []

    response = requests.get('https://www.ig.com.br/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')

    # principais
    main = site.find('div', attrs={'class': 'destaque_containerText'})

    principais = main.find_all('a', attrs={'data-tb-region-item': 'taboola-region'})
    for noticia in principais:
    # Título
        titulo = noticia.text


        lista_noticias.append(["ig_home",titulo, '', noticia['href']])

    # HTML da notícia
    noticias = site.findAll('a', attrs={'class': 'title'})

    for noticia in noticias:
        
    # Título
        titulo = noticia.text


        lista_noticias.append(["ig_home",titulo, '', noticia['href']])


    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])



    #o site utiliza javascript, por isso é preciso utilizar a biblioteca selenium
    # Refs: https://medium.com/data-hackers/web-scraping-com-python-para-pregui%C3%A7osos-unindo-beautifulsoup-e-selenium-parte-2-8cfebf4f34e
    # https://blog.4linux.com.br/web-scraping-python-selenium-e-beautifulsoup/

    lista_noticias=[]



    url = ("https://ultimosegundo.ig.com.br/")

    #acessando a url
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)

    #Procurando o elemento da página
    element = driver.find_element_by_class_name('ultimosegundo')

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    noticias = site.findAll('a', attrs={'class': 'destaque-item-link'})

    for noticia in noticias:
    # Título
        titulo = noticia.find("img")['title']


        lista_noticias.append(["ig_ultimas" ,titulo, '', noticia['href']])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    df = pd.concat([news1,news2])

    df.to_excel('ig.xlsx', index=False)

In [16]:
def antagonista ():
    
    lista_noticias = []
    url = ("https://www.oantagonista.com/brasil/")

    #acessando a url
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)


    elem = driver.find_element_by_xpath("/html/body")


    #loading pages (https://stackoverflow.com/questions/21006940/how-to-load-all-entries-in-an-infinite-scroll-at-once-to-parse-the-html-in-pytho)
    no_of_pagedowns = 5

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        no_of_pagedowns-=1


    #Procurando o elemento da página
    element = driver.find_element_by_class_name('container_home_left')

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    # HTML da notícia
    noticias = site.findAll('div', attrs={'class': 'post_header'})

    for noticia in noticias:
        # Título
        titulo = noticia.find('a')['title']
        subtitulo = noticia.find('p').text

        lista_noticias.append(["antagonista_brasil",titulo, subtitulo, noticia.find('a')['href']])


    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])
    news1.to_excel('antagonista.xlsx', index=False)

In [17]:
def oglobo():
    lista_noticias = []

    response = requests.get('https://oglobo.globo.com/ultimas-noticias/')

    content = response.content

    site = BeautifulSoup(content, 'html.parser')


    principal = site.find('h1', attrs={'class': 'teaser__title headline__title'})
    titulo = principal.text.strip()
    lista_noticias.append(["oglobo_ultimas",titulo, '', principal.find('a')['href']])

    # HTML da notícia
    noticias = site.findAll('h1', attrs={'class': 'teaser__title teaser__title'})

    for noticia in noticias:
    # Título
        titulo = noticia.text.strip()

        lista_noticias.append(["oglobo_ultimas",titulo, '', noticia.find('a')['href']])




    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])


    lista_noticias = []
    url = "https://oglobo.globo.com/rss.xml?completo=true"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('item')

    for noticia in noticias:
    # Título
        titulo = noticia.find('title')

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = re.sub(r'<.+?>', '', noticia.find('description').text).strip()

        if (subtitulo):

            lista_noticias.append(["oglobo_rss", titulo.text, subtitulo, noticia.find("link").text])
        else:
            lista_noticias.append(["oglobo_rss" ,titulo.text, '', noticia.find("link").text])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])


    df = pd.concat([news1 , news2])
    df.to_excel('oglobo.xlsx', index=False)

In [18]:
def ny_times():
    lista_noticias = []
    url = "https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('item')

    for noticia in noticias:
      # Título
        titulo = noticia.find('title')

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = re.sub(r'<.+?>', '', noticia.find('description').text).strip()

        if (subtitulo):

            lista_noticias.append(["ny_times", titulo.text, subtitulo, noticia.find("link").text])
        else:
            lista_noticias.append(["ny_times" ,titulo.text, '', noticia.find("link").text])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])


    news2.to_excel('ny_times.xlsx', index=False)

In [19]:
def washington():
    #o site utiliza javascript, por isso é preciso utilizar a biblioteca selenium
    # Refs: https://medium.com/data-hackers/web-scraping-com-python-para-pregui%C3%A7osos-unindo-beautifulsoup-e-selenium-parte-2-8cfebf4f34e
    # https://blog.4linux.com.br/web-scraping-python-selenium-e-beautifulsoup/

    lista_noticias=[]



    url = ("https://www.washingtonpost.com/")

    #acessando a url
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)

    #Procurando o elemento da página
    element = driver.find_element_by_xpath("/html/body/div[1]/div/main")

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    noticias = site.findAll('div', attrs={'class': 'card-top card-text'}) 

    for noticia in noticias:
    # Título
        try:
            titulo = noticia.find("span").text
        except:
            continue


        lista_noticias.append(["washington" ,titulo, '',noticia.find("a")['href']])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])
    news2 = news2[news2['Título'].str.len() > 20]

    news2.to_excel('washington.xlsx', index=False)

In [20]:
def cnn_br():
    
    lista_noticias=[]



    url = ("https://www.cnnbrasil.com.br/ultimas-noticias/")

    #acessando a url
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)

    #Procurando o elemento da página
    element = driver.find_element_by_xpath("/html/body/div[2]/main/div/div[1]")

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    noticias = site.findAll('a', attrs={'class': 'home__list__tag'}) 

    for noticia in noticias:
    # Título
        titulo = noticia.text.strip()


        lista_noticias.append(["cnn_br_ultimas" ,titulo, '',noticia['href']])

    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])


    
    
    
    
    lista_noticias=[]



    url = ("https://www.cnnbrasil.com.br/")

    #acessando a url
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)

    #Procurando o elemento da página
    element = driver.find_element_by_xpath("/html/body/div[2]")

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    noticias = site.findAll('a', attrs={'class': 'home__post'}) 

    for noticia in noticias:
    # Título
        titulo = noticia.text.strip()


        lista_noticias.append(["cnn_br_home" ,titulo, '',noticia['href']])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])

    df = pd.concat([news1 , news2])
    df.to_excel('cnn_br.xlsx', index=False)

In [21]:
def cnn_en():
    lista_noticias=[]



    url = ("https://edition.cnn.com/")

    #acessando a url
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)

    #Procurando o elemento da página
    element = driver.find_element_by_xpath("/html/body")

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    noticias = site.findAll('div', attrs={'class': 'cd__content'}) 

    for noticia in noticias:
    # Título
        titulo = noticia.text
        link = "https://edition.cnn.com"+noticia.find('a')['href']

        lista_noticias.append(["cnn_en" ,titulo, '',link])

    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])
    news1 = news1[news1['Título'].str.len() > 30]
    news1.to_excel('cnn_en.xlsx', index=False)

In [22]:
def bbc_br():
    
    lista_noticias = []
    url = "http://www.bbc.co.uk/portuguese/index.xml"
    response = requests.get(url)
    site = BeautifulSoup(response.content, features='xml')
    noticias = site.findAll('item')

    for noticia in noticias:
      # Título
        titulo = noticia.find('title')

        # Subtítulo: div class="feed-post-body-resumo"
        subtitulo = re.sub(r'<.+?>', '', noticia.find('description').text).strip()

        if (subtitulo):

            lista_noticias.append(["bbc_br", titulo.text, subtitulo, noticia.find("link").text])
        else:
            lista_noticias.append(["bbc_br" ,titulo.text, '', noticia.find("link").text])

    news2 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link'])


    news2.to_excel('bbc_br.xlsx', index=False)

In [23]:
def bbc_en():
    
    lista_noticias = []

    url = ("https://www.bbc.com/news")

    #acessando a url
    driver.get(url)

    #Esperando o carregamento
    time.sleep(5)

    #Procurando o elemento da página
    element = driver.find_element_by_xpath("/html/body")

    #Carregando o html
    html = element.get_attribute("innerHTML")

    #Utilizando o BeautifulSoup para acessar os dados
    site = BeautifulSoup(html, "html.parser")


    principais = site.findAll('a', attrs={'class': 'gs-c-promo-heading gs-o-faux-block-link__overlay-link gel-paragon-bold nw-o-link-split__anchor'}) 


    for noticia in principais:
        try:

        # Título
            titulo = noticia.text
            link = "https://www.bbc.com"+noticia['href']

            lista_noticias.append(["bbc_en" ,titulo, '',link])
        except:
            break



    noticias = site.findAll('a', attrs={'class':"gs-c-promo-heading gs-o-faux-block-link__overlay-link gel-pica-bold nw-o-link-split__anchor"})

    for noticia in noticias:
    # Título
        titulo = noticia.text
        link = "https://www.bbc.com"+noticia['href']

        lista_noticias.append(["bbc_en" ,titulo, '',link])

    news1 = pd.DataFrame(lista_noticias, columns=["fonte",'Título', 'Subtítulo', 'Link']).drop_duplicates(subset= "Título",keep='last')
    news1.to_excel('bbc_en.xlsx', index=False)

In [31]:
if __name__ == '__main__':
    p1 = Process(target=g1())
    p2 = Process(target=r7())
    p3 = Process(target=folha())
    p4 = Process(target=estadao())
    p5 = Process(target=metro())
    p6 = Process(target=globo())
    p7 = Process(target=uol())
    p8 = Process(target=terra())
    p9 = Process(target=msn())
    p10 = Process(target=buzz())
    p11 = Process(target=veja())
    p12 = Process(target=ig())
    p13 = Process(target=antagonista())
    p14 = Process(target=oglobo())
    p15 = Process(target=ny_times())
    p16 = Process(target=washington())
    p17 = Process(target=cnn_br())
    p18 = Process(target=cnn_en())
    p19 = Process(target=bbc_br())
    p20 = Process(target=bbc_en())
    p1.start() 
    p2.start() 
    p3.start() 
    p4.start() 
    p5.start() 
    p6.start() 
    p7.start() 
    p8.start() 
    p9.start() 
    p10.start()
    p11.start()
    p12.start()
    p13.start()
    p14.start()
    p15.start()
    p16.start()
    p17.start()
    p18.start()
    p19.start()
    p20.start()
